In [2]:
!pip install -q transformers einops accelerate bitsandbytes langchain-huggingface langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Sat Nov 23 03:17:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [4]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0}) # templerature 0 means you need exact answers. bigger value means accept random answers.

In [ ]:
from langchain import PromptTemplate

template = """
You are an intelligent chatbot. Help the following question with brilliant answers.
Question: {question}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = prompt | llm

question = "Explain what is Artificial Intellience as Nursery Rhymes "

print(llm_chain.invoke(question))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



You are an intelligent chatbot. Help the following question with brilliant answers.
Question: Explain what is Artificial Intellience as Nursery Rhymes 
Answer: 

AI is just like a big toy,
That can learn and do many tricks!
It has the power to think,
And can make decisions quick.
It can learn things in ways we never could,
And even help you do your sums.
It does things faster than you could ever do,
And can process data like nobody's business.
So, if you want to be more intelligent,
And learn things faster than you can imagine,
Just embrace the power of AI -
And you'll be amazed at its infinite knowledge!


# Multiple Input variables in Prompt Template

In [5]:
from langchain import PromptTemplate

template = """
What is a good name for a {company} that makes {product}?
"""
prompt = PromptTemplate(template=template, input_variables=["company", "product"])

llm_chain = prompt | llm

question = "Explain what is Artificial Intellience as Nursery Rhymes "

print(llm_chain.invoke({'product': 'laptops', 'company':'computer hardware'}))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



What is a good name for a computer hardware that makes laptops?
A good name for a computer hardware that makes laptops could be Intel. It could also be HP, AMD, or Toshiba.
